In [1]:
!pip3 install pystan prophet

  Using cached prophet-1.0.1.tar.gz (65 kB)
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=665003 sha256=7f7c044808cb85c4b9bb7a89979107adfc11d9de56acf159ba5f2ea1a9da102c
  Stored in directory: /Users/yevheniyavytruchenko/Library/Caches/pip/wheels/26/0b/d9/6583b6a639c0ff50333d558826d4366c7739bd6a7450b80fec
Successfully built prophet


In [4]:
import pandas as pd
import prophet

In [3]:
df = pd.read_csv('dataset.csv')